In [1]:
import sys
sys.path.append("../")
import os
import vqe_experiment as ve
import numpy as np

In [2]:
# define Hamiltonian: list of coefficients, list of Paulis, initial bitstring

# here: molecular Hamiltonian, parity mapping, two-qubit reduction
atom_string = "H 0 0 0; H 0 0 2.0"              # PySCF atom string
num_orbitals = 1                                 # number of spatial orbitals
coeffs, paulis, HF_bitstring = ve.molecule(atom_string, new_num_orbitals=num_orbitals)

The original qubit operator only contains 2 qubits! Skipping the requested two-qubit reduction!
The original qubit operator only contains 2 qubits! Skipping the requested two-qubit reduction!


In [3]:
coeffs, paulis, HF_bitstring

(array([-0.65155633,  0.26209531,  0.26209531,  0.1273657 ,  0.26458861]),
 array(['II', 'ZI', 'ZZ', 'IZ', 'II'], dtype='<U2'),
 '10')

In [4]:
# number of qubits
N = len(paulis[0])
N

2

In [5]:
# configure underlying VQE structure, passed as keyword argument dictionary

# a function which returns a parameterized Qiskit circuit (ansatz) and the number of parameters
ansatz_func = ve.efficientsu2_full
ansatz_reps = 1
# if True, the initial HF_bitstring will be appended after ansatz (layer of X gates where there are 1s)
# if False, HF_bitstring will be initialized before ansatz
init_last = False

vqe_kwargs = {
    "ansatz_func": ansatz_func,
    "ansatz_reps": ansatz_reps,
    "init_last": init_last,
    "HF_bitstring": HF_bitstring
}

In [6]:
ansatz, num_params = ansatz_func(N, ansatz_reps)
print(num_params)

8


In [7]:
ansatz.draw(fold=-1)

┌──────────┐┌──────────┐ ░       ░ ┌──────────┐┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├─░───■───░─┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├
     ├──────────┤├──────────┤ ░ ┌─┴─┐ ░ ├──────────┤├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├─░─┤ X ├─░─┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├
     └──────────┘└──────────┘ ░ └───┘ ░ └──────────┘└──────────┘

CAFQA parameters are represented by a list $[x_0, x_1, x_2, ...]$ where $x_i$ parametrizes the $i$-th gate in the ansatz and can only take the discrete values $\{0,1,2,3\}$. Typically the parameterized gates are rotation gates, $R_y$ or $R_z$, so that $x_i$ corresponds to a rotation angle $x_i \cdot \pi/2$.

In [8]:
# configure algorithm

budget = 100                                                 # number of iterations
guess = list(map(int, np.random.randint(0, 4, num_params)))  # initial param guess
guess

[2, 2, 0, 3, 3, 1, 3, 1]

In [9]:
# get VQE circuit guess parameters (* pi/2)
qc = ve.vqe_circuit(
    N, 
    np.array(guess)*np.pi/2, 
    paulis, 
    ansatz_func=ansatz_func, 
    ansatz_reps=ansatz_reps, 
    init_last=init_last,
    HF_bitstring=HF_bitstring
    )

In [10]:
qc.draw(fold=-1)

┌───┐   ┌───────┐  ┌───────┐ ░       ░ ┌──────────┐┌──────────┐
q1_0: ──┤ X ├───┤ Ry(π) ├──┤ Rz(0) ├─░───■───░─┤ Ry(3π/2) ├┤ Rz(3π/2) ├
      ┌─┴───┴─┐┌┴───────┴─┐└───────┘ ░ ┌─┴─┐ ░ ├─────────┬┘├─────────┬┘
q1_1: ┤ Ry(π) ├┤ Rz(3π/2) ├──────────░─┤ X ├─░─┤ Ry(π/2) ├─┤ Rz(π/2) ├─
      └───────┘└──────────┘          ░ └───┘ ░ └─────────┘ └─────────┘ 
c0: 2/═════════════════════════════════════════════════════════════════

In [11]:
# configure output

save_dir = "cafqa_example_out/"
os.system(f"mkdir -p {save_dir}")
# the output files are relative to save_dir
result_file = "result.txt"
loss_file = "cafqa_loss.txt"
params_file = "cafqa_params.txt"

In [12]:
# run CAFQA

# hypermapper changes the standard output to write to log files, so it needs to be
# changed back after CAFQA (otherwise printing breaks)

stdout = sys.stdout
cafqa_energy, cafqa_params = ve.run_cafqa(
    n_qubits=N,
    coeffs=coeffs,
    paulis=paulis,
    param_guess=list(guess),
    budget=budget,
    save_dir=save_dir,
    loss_file=loss_file,
    params_file=params_file,
    vqe_kwargs=vqe_kwargs
)
sys.stdout = stdout

Design of experiment phase, number of new doe samples = 100 .......
Loss computed by CAFQA VQE is -0.38696772751525116, in 0.01916920799703803 s.
Loss computed by CAFQA VQE is -0.12487241258091292, in 0.021314665995305404 s.
Loss computed by CAFQA VQE is -0.2596020244086766, in 0.013721415991312824 s.
Loss computed by CAFQA VQE is -0.5143334306218257, in 0.020783542000572197 s.
Loss computed by CAFQA VQE is -0.5143334306218257, in 0.014188417000696063 s.
Loss computed by CAFQA VQE is -0.38696772751525116, in 0.01691266699344851 s.
Loss computed by CAFQA VQE is -0.38696772751525116, in 0.035942625007010065 s.
Loss computed by CAFQA VQE is -0.6490630424495893, in 0.019708000007085502 s.
Loss computed by CAFQA VQE is -0.12487241258091292, in 0.018072000006213784 s.
Loss computed by CAFQA VQE is -0.12487241258091292, in 0.019423125006142072 s.
Loss computed by CAFQA VQE is -0.38696772751525116, in 0.020441667002160102 s.
Loss computed by CAFQA VQE is -0.38696772751525116, in 0.019204917000

AttributeError: Can't pickle local object 'Space.__init__.<locals>.<lambda>'